In [1]:
import glob,os
import tifffile as tiff
import warnings
warnings.filterwarnings("ignore")
import shutil
import pandas as pd
import numpy as np
from PIL import Image

In [2]:
DIR = '/Users/damoncrockett/Desktop/preprocessed_textures/'

In [3]:
allfiles = glob.glob(os.path.join(DIR,'*.tif'))

In [4]:
# These are all readable (I had to update tifffile to a newer version bc of PACKBITS issue)

"""
for item in allfiles:
    try:
        tiff.imread(item)
    except Exception as e:
        print(e)
"""

'\nfor item in allfiles:\n    try:\n        tiff.imread(item)\n    except Exception as e:\n        print(e)\n'

In [5]:
# Scales at which to compute relative areas
#   - logarithmically spaced
#   - fibonacci sequence
scale = [1, 2, 3, 5, 8, 13, 21, 34]

# Scaling factor used in normalization step (found by trial)
gain = 250

# Extracted image will be NxN = 1024x1024 (middle chunk of the image)
N = 1024

## functions

In [6]:
def crop_array(array,N):
    
    h, w = array.shape
    
    left  = w/2 - N/2
    upper = h/2 - N/2
    right = w/2 + N/2
    lower = h/2 + N/2
    return array[int(upper):int(lower),int(left):int(right)]

In [7]:
def process_images(allfiles, scale, gain, N):
    
    # Allocate space for relative areas
    A = np.zeros((len(allfiles), len(scale)))

    for i,filename in enumerate(allfiles):
        # -------------------------------------------------
        # Christoph Gohlke's tifffile module -> imread tifs
        # -------------------------------------------------
        # Normalize array by 2^16

        tif_array = np.array(tiff.imread(filename),dtype=np.float64)/(2**16)
        print("texture processing",i,"of",len(allfiles),filename)


        # Crop array to extract middle 1024x1024 portion of image
        tif_array = crop_array(tif_array,N)

        # Image T has already been cropped (middle 1024x1024 portion extracted) and normalized by 2^16
        # These are all grayscale already, so no need to convert
        
        T = tif_array

        # Normalize by total intensity
        T = (gain*T)/(np.sum(T))*(N**2)

        # Compute relative areas for each image, at each scale
        for j in range(len(scale)):

            # Subtract current scale from each element in vector
            idx = np.arange(N - scale[j])

            A[i][j] = np.sum( ( scale[j]
            * np.sqrt( ( np.square( T[np.ix_(idx + scale[j], idx)]
            - T[np.ix_(idx, idx)])
            + (np.square( T[np.ix_(idx, idx + scale[j])]
            - T[np.ix_(idx, idx)]  ))
            + (scale[j]**2))  ) / 2  ) + (scale[j]
            * np.sqrt(  ( np.square( T[np.ix_(idx + scale[j], idx+scale[j])]
            - T[np.ix_(idx, idx + scale[j])])
            + (np.square( T[np.ix_(idx + scale[j], idx
            + scale[j])] - T[np.ix_(idx + scale[j], idx)]  ))
            + (scale[j]**2))  ) / 2)  ) / ((N-scale[j]) * scale[j])**2

    return A

### Get texture descriptors

In [10]:
A = process_images(allfiles, scale, gain, N)
    
index_col = [item for item in range(A.shape[0])]
cols = [str(item) for item in range(A.shape[1])]
X = pd.DataFrame(index=index_col,columns=cols)

for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        X.loc[i,str(j)] = A[i][j]

texture processing 0 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2599_23_vt.tif
texture processing 1 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1327_64_rt.tif
texture processing 2 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2597_1_vr.tif
texture processing 3 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2602_24_vt.tif
texture processing 4 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1295_26_rt.tif
texture processing 5 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1294_epreuve_8_rt.tif
texture processing 6 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1249_4_vt.tif
texture processing 7 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1403_33_rr.tif
texture processing 8 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1301_45_vt.tif
texture processing 9 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2603_essai_vt.tif
texture processing 10 of 860 /Users/damoncrockett/De

texture processing 86 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2596_3_vt.tif
texture processing 87 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1256_essai_japon_rr.tif
texture processing 88 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1299_19_vr.tif
texture processing 89 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2599_5_RT.tif
texture processing 90 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1301_5_vr.tif
texture processing 91 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2594_3_rr.tif
texture processing 92 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1368_6_vt.tif
texture processing 93 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2605_19_rr.tif
texture processing 94 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1324_72_vr.tif
texture processing 95 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2608_17_rt.tif
texture processing 96 of 860 /Users/damoncrock

texture processing 171 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1296_epreuve_6_vt.tif
texture processing 172 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1363_14_vt.tif
texture processing 173 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2607_8_rr.tif
texture processing 174 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1361_16_rr.tif
texture processing 175 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1335_vr.tif
texture processing 176 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1253_essai_b_rt.tif
texture processing 177 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2601_16_rt.tif
texture processing 178 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1367_21_vt.tif
texture processing 179 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1313_13_rr.tif
texture processing 180 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2610_11_rr.tif
texture processing 181 of 860 /U

texture processing 256 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1362_20_vt.tif
texture processing 257 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1300_83_rr.tif
texture processing 258 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1403_4_vr.tif
texture processing 259 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1256_essai_rr.tif
texture processing 260 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1387_18_vr.tif
texture processing 261 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1295_41_rr.tif
texture processing 262 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1296_28_treated_vr.tif
texture processing 263 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1358_23_vr.tif
texture processing 264 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2595_6_vt.tif
texture processing 265 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1294_29_vr.tif
texture processing 266 of 860 /

texture processing 341 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1387_22_vt.tif
texture processing 342 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1348_18_vr.tif
texture processing 343 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1301_essai_japon_vr.tif
texture processing 344 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1368_1_rr.tif
texture processing 345 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1362_17_rr.tif
texture processing 346 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1294_4_vt.tif
texture processing 347 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1327_49_vt.tif
texture processing 348 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2595_20_rt.tif
texture processing 349 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1403_13_rr.tif
texture processing 350 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1298_50_vr.tif
texture processing 351 of 860 /Us

...

texture processing 510 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1327_68_vt.tif
texture processing 511 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2592_essai_a_rt.tif
texture processing 512 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1403_32_rr.tif
texture processing 513 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2595_12_vt.tif
texture processing 514 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1249_5_vt.tif
texture processing 515 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2607_epreuve_4_rt.tif
texture processing 516 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1403_21_vr.tif
texture processing 517 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1358_2_vr.tif
texture processing 518 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2603_3_vt.tif
texture processing 519 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1313_essai_rr.tif
texture processing 520 of 86

texture processing 595 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2595_essai_rt.tif
texture processing 596 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1295_essai_vr.tif
texture processing 597 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1327_41_rr.tif
texture processing 598 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2600_10_vt.tif
texture processing 599 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2602_epreuve_3_vt.tif
texture processing 600 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2603_21_vt.tif
texture processing 601 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1296_32_rr.tif
texture processing 602 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1299_50_rr.tif
texture processing 603 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2589_essai_vr.tif
texture processing 604 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1365_16_rt.tif
texture processing 605 o

texture processing 680 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1298_essai_vr.tif
texture processing 681 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2597_essai_vt.tif
texture processing 682 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2591_16_rt.tif
texture processing 683 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1327_29_vt.tif
texture processing 684 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2601_9_rr.tif
texture processing 685 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1364_23_vt.tif
texture processing 686 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1313_4_rt.tif
texture processing 687 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1367_epeuve_2_vr.tif
texture processing 688 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2605_10_rt.tif
texture processing 689 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2597_21_vr.tif
texture processing 690 of 860 

texture processing 765 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2599_essai_RT.tif
texture processing 766 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1348_21_vt.tif
texture processing 767 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1361_12_vt.tif
texture processing 768 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2596_essai_vt.tif
texture processing 769 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1299_essai_vr.tif
texture processing 770 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2606_16_vr.tif
texture processing 771 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1367_24_vr.tif
texture processing 772 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2608_6_rt.tif
texture processing 773 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1324_40_rr.tif
texture processing 774 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1295_20_vr.tif
texture processing 775 of 860 /U

texture processing 850 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1366_6_vr.tif
texture processing 851 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2600_19_rr.tif
texture processing 852 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1299_24_vt.tif
texture processing 853 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1331_essai_a_rt.tif
texture processing 854 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1265_4_vt.tif
texture processing 855 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1300_34_rr.tif
texture processing 856 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1301_essai_b_rt.tif
texture processing 857 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1313_epreuve_10_rr.tif
texture processing 858 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/1296_9_treated_vt.tif
texture processing 859 of 860 /Users/damoncrockett/Desktop/preprocessed_textures/2608_epreuve_1_vr.tif


In [11]:
import sys
sys.path.append('/Users/damoncrockett/ivpytest/src/')
from ivpy.extract import norm

In [13]:
X = norm(X)

In [14]:
X['localpath'] = allfiles

In [16]:
X.to_csv('/Users/damoncrockett/Desktop/pasfa.csv',index=False)